# Enrollment Simulations

This notebook uses the cleaned data from `data/data_cleaning.ipynb` and the trained models from `models/model_training.ipynb` to simulate student enrollment when schools use model predictions as noisy estimates of student quality. The data from simulations is saved and can be visualized in `simulation_plots.ipynb.`


In [1]:
import numpy as np
import pandas as pd
import pickle
import os
from datetime import datetime
from tqdm.notebook import trange

# for parallelism
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
import multiprocessing

master_seed = 1234
master_rng = np.random.default_rng(seed = master_seed)

# Loading Data
We first load the cleaned student data generated in `data/data_cleaning.ipynb`

In [2]:
train_df = pd.read_csv("../data/no_gpa_3_cleaned_data.csv")

We use `GPA_3` as our target variable, which is an indicator that a student's first-year college GPA was greater than 3. 

The cleaned dataset contains a few additional features (sex, race, GPA, student id) that we do not use in training, but retain for demographic information in later experiments. 

The code below constructs the list of training features.

In [3]:
protected_features = ["F1SEX", "F1RACE", "F3TZYR1GPA", "id"]
outcome = "GPA_3"
# features are all the columns except protected features and outcome
features = list(train_df.columns)
for feature in protected_features:
    features.remove(feature)
features.remove(outcome)

In [4]:
train_X = train_df[features]
train_y = train_df[outcome]

We set our applicant pool to be four copies of the entire student dataset, for a toal pool of 30,112 students. 

In [5]:
enroll_X = pd.concat([train_X, train_X, train_X, train_X], axis=0).reset_index(drop=True)
enroll_y = pd.concat([train_y, train_y, train_y, train_y], axis=0).reset_index(drop=True)
extended_feature_enroll_X = pd.concat([train_df, train_df, train_df, train_df], axis=0).reset_index(drop=True)

# Shared Helper Functions

There are a number of helper functions used throughout our different enrollment simulations. Most importantly, we implement the deferred acceptance algorithm, which is used to determine final enrollment outcomes based on noisy rankings of schools. 

## Deferred Acceptance Algorithm

In [6]:
def DA(school_prefs, student_prefs, capacities):
    '''
    Given school and student rankings and a list of capacities over each school,
    runs the deferred acceptance algorithm and outputs the final matching. 

    school_prefs: an array with num_schools rows and num_students columns. school_prefs[i][j] 
    gives the index of school i's jth favorite student. 

    student prefs: an array with num_students rows and num_schools columns. student_prefs[i][j]
    gives the index of student i's jth favorite school. 
    '''
    
    num_students = len(student_prefs)
    num_schools = len(school_prefs)

    # will store the enrolled school index or -1 for each student.
    enrolled = np.array([-1 for _ in range(num_students)]) 

    # keeps track of where each school is in their ranking of students as they propose
    curr_t = np.array([capacities[i] for i in range(num_schools)])

    # stores the number of students currently enrolled at each school.
    curr_enrolled = np.array([0 for _ in range(num_schools)])

    # initial enrollment: 
    for i in range(num_schools):
        for j in range(curr_t[i]):
            curr_student = school_prefs[i][j]
            if enrolled[curr_student] == -1:
                enrolled[curr_student] = i
                curr_enrolled[i] += 1
            elif np.where(student_prefs[curr_student]==i) < np.where(student_prefs[curr_student] == enrolled[curr_student]):
                curr_enrolled[enrolled[curr_student]] -= 1
                enrolled[curr_student] = i
                curr_enrolled[i] += 1
    
    while not np.array_equal(curr_enrolled, capacities):
        for i in range(num_schools):
            if curr_enrolled[i] != capacities[i]:
                assert(curr_enrolled[i] < capacities[i])
                curr_student = school_prefs[i][curr_t[i]]
                curr_t[i] += 1

                if enrolled[curr_student] == -1:
                    enrolled[curr_student] = i
                    curr_enrolled[i] += 1
                elif np.where(student_prefs[curr_student]==i) < np.where(student_prefs[curr_student] == enrolled[curr_student]):
                    curr_enrolled[enrolled[curr_student]] -= 1
                    enrolled[curr_student] = i
                    curr_enrolled[i] += 1
    return enrolled

## Helpers for Generating Student Features

In [7]:
def generate_uniform_preferences(num_students, num_schools, rng=master_rng):
    '''
    Generate num_students different rankings over num_schools options, each drawn uniformly 
    from all possible permutations over school choices. 
    '''
    assert(num_students > 0 and num_schools > 0)
    return np.array([rng.permutation(num_schools) for _ in range(num_students)])
    

## Helpers for Getting School Estimates

In [8]:
def load_model(model_type, model_num, file_path="../models/pretrained_models"):
    '''
    Load a pretrained model from a specified file path.
    '''
    with open(f'{file_path}/{model_type}/{model_type}_model_{model_num}.pkl', 'rb') as f:
        return pickle.load(f)

In [9]:
def get_estimates_from_models(model_indices, model_type, students):
    '''
    Given a list of model indices, the model type, and a dataframe of student data, output 
    a list of model-predicted estimates for the students using the model at each provided index.
    '''
    ests = []
    for i in model_indices:
        m = load_model(model_type, i)
        estimates = m.predict_proba(students)[:, 1]
        ests.append(estimates)
    return np.array(ests)
    

In [10]:
def get_rankings_from_estimates(estimates):
    '''
    Given a list of estimates for a pool of students, use that to return a ranking over 
    students as a list of student indices in descending order by estimates. If the input 
    is multiple estimates, the output returns a ranking for each estimate. 
    '''
    rankings = []
    for est in estimates:
        ranking = np.argsort(est)[::-1]
        rankings.append(ranking)
    return np.array(rankings)


In [11]:
def set_up_school_features(num_schools, estimate_type, students, rng=master_rng, num_models = 200, model_type = 'log'):
    '''
    Assign noisy estimates sources to schools according to monoculture or polyculture 
    and construct each school's ranking over students by estimate.
    '''
    assert(estimate_type in ['mono', 'poly'])

    if estimate_type == 'poly':
        model_indices = rng.choice(range(num_models), num_schools, replace=False)
    else:
        model_indices = rng.choice(range(num_models), 1)

    estimates = get_estimates_from_models(model_indices, model_type, students)
    rankings = get_rankings_from_estimates(estimates)

    if estimate_type == 'mono':
        rankings = np.array([rankings[0]] * num_schools)
    
    return rankings

In [12]:
def interleave_rankings(school_rankings, group_membership):
    '''
    Given a list of school rankings over students and a group membership vector that 
    splits the students into two groups, create a new set of interleaved rankings where 
    for each school, if the induced ranking over group 0 students is (i_1, i_2, ...) and 
    the induced ranking over group 1 students is (j_1, j_2, ...), the outputted interleaved
    ranking consists of (i_1, j_1, i_2, j_2, ...).
    '''
    interleaved_rankings = []
    for r in school_rankings:
        r = np.array(r)
        group_membership = np.array(group_membership)
        group_0_indices = r[group_membership[r] == 0]
        group_1_indices = r[group_membership[r] == 1]
        
        combined_ranking = []

        curr_index = 0
        
        while (curr_index < len(group_0_indices)) or (curr_index < len(group_1_indices)):
            if curr_index < len(group_0_indices):
                combined_ranking.append(group_0_indices[curr_index])
            if curr_index < len(group_1_indices): 
                combined_ranking.append(group_1_indices[curr_index])
            curr_index += 1
        interleaved_rankings.append(combined_ranking)
    
    return interleaved_rankings



## End-to-End run of single Enrollment Simulation

In [13]:
def run_one_enrollment(num_schools, students, capacity, model_type = 'log', rng=master_rng, num_models = 200, groups = None, group_fraction = None):
    assert (capacity <= len(students))
    if groups not in [None, 'synth', 'sport', 'legacy']:
        raise RuntimeError(f"Group setting {groups} not supported.")
    # school capacities
    capacities = np.array([capacity//num_schools for _ in range(num_schools)])

    # school-side rankings over students (from model predictions)
    poly_rankings = set_up_school_features(num_schools, 'poly', students, model_type = model_type, rng=rng, num_models=num_models)
    mono_rankings = set_up_school_features(num_schools, 'mono', students, model_type = model_type, rng=rng, num_models=num_models)

    # student-side rankings over schools (generated uniformly at random)
    student_rankings = generate_uniform_preferences(len(students), num_schools, rng=rng)

    group_membership = None

    if groups == 'synth':
        assert(group_fraction is not None)
        group_membership = np.zeros(len(students), dtype=int)
        group_size = int(np.floor(group_fraction * len(students)))
        assert(group_size >= capacity/2)
        assert(len(students) - group_size >= capacity/2)
        group_indices = rng.choice(range(len(students)), group_size, replace = False)
        group_membership[group_indices] = 1
        assert(np.sum(group_membership) == group_size )

    
    if groups == 'sport':
        group_membership = np.array(students['F1S26B_3'].values)
        assert((np.sum(group_membership) >= capacity/2) & (np.sum(1-group_membership) >= capacity/2) )

    if groups in ['sport', 'synth']:   
        poly_rankings = interleave_rankings(poly_rankings, group_membership)
        mono_rankings = interleave_rankings(mono_rankings, group_membership)

    poly_enrollment = DA(poly_rankings, student_rankings, capacities)
    mono_enrollment = DA(mono_rankings, student_rankings, capacities)

    if groups == 'legacy':
        # group membership stores students' top choice schools.
        first_choice_rankings = student_rankings[:, 0]
        group_membership = first_choice_rankings

    return poly_enrollment, mono_enrollment, group_membership

In [14]:
def enroll_and_calc_stats(num_schools, students, student_labels, capacity, model_type, iteration, groups = None, group_fraction = None, seed = None, num_models = 200, identifier=None):
    if seed is not None:
        rng = np.random.default_rng(seed)
    else:
        rng = master_rng

    if groups not in [None, 'synth', 'sport', 'legacy']:
        raise RuntimeError(f"Group setting {groups} not supported.")
        
    poly_enrollment, mono_enrollment, group_membership = run_one_enrollment(num_schools, students, capacity, groups = groups, group_fraction = group_fraction, model_type = model_type, rng=rng)

    poly_students = np.where(poly_enrollment != -1)[0]
    mono_students = np.where(mono_enrollment != -1)[0]
    poly_not_mono_students = np.where((poly_enrollment != -1) & (mono_enrollment == -1))[0]
    mono_not_poly_students = np.where((poly_enrollment == -1) & (mono_enrollment != -1))[0]

    if groups is None:
        poly_mean = np.mean(student_labels[poly_students])
        mono_mean = np.mean(student_labels[mono_students])
        poly_not_mono_mean = np.mean(student_labels[poly_not_mono_students])
        mono_not_poly_mean = np.mean(student_labels[mono_not_poly_students])

        if identifier is not None:
            return (identifier, (poly_mean, mono_mean, poly_not_mono_mean, mono_not_poly_mean))

        return (poly_mean, mono_mean, poly_not_mono_mean, mono_not_poly_mean)
    
    if groups in ['synth', 'sport']:
        poly_group_1_prop = np.mean(group_membership[poly_students])
        mono_group_1_prop = np.mean(group_membership[mono_students])
        p_not_m_group_1_prop = np.mean(group_membership[poly_not_mono_students])
        m_not_p_group_1_prop = np.mean(group_membership[mono_not_poly_students])

        if identifier is not None:
            return (identifier, (poly_group_1_prop, mono_group_1_prop, p_not_m_group_1_prop, m_not_p_group_1_prop))
        
        return (poly_group_1_prop, mono_group_1_prop, p_not_m_group_1_prop, m_not_p_group_1_prop)
    
    if groups == 'legacy':
        # in this case, group membership stores student first-choice schools. 
        # we want to keep track of the enrollment quality for students enrolled at their top choice vs all students
        poly_mean = np.mean(student_labels[poly_students])
        mono_mean = np.mean(student_labels[mono_students])
        poly_first_choice_students = np.where((poly_enrollment != -1) & (poly_enrollment == group_membership))[0]
        mono_first_choice_students = np.where((mono_enrollment != -1) & (mono_enrollment == group_membership))[0]
        poly_first_choice_mean = np.mean(student_labels[poly_first_choice_students])
        mono_first_choice_mean = np.mean(student_labels[mono_first_choice_students])

        if identifier is not None:
            return (identifier, (poly_mean, mono_mean, poly_first_choice_mean, mono_first_choice_mean))

        return (poly_mean, mono_mean, poly_first_choice_mean, mono_first_choice_mean)
        
    
    

# Enrollment Simulation 1 of 3: Overall Effects of Polyculture vs Monoculture

This cell generates and saves simulation data tracking the difference in average student quality between enrolled classes in monoculture and polyculture. See *Effects on Average Enrolled Student Quality* in Section 5.2.2 of the Arxiv manuscript. Results can be visualized in `simulation_plots.ipynb`.

In [15]:
schools = [1, 2, 4, 8]
capacity = 10000
num_its = 2000

os.makedirs('saved_runs/avg_effects', exist_ok = True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
prefix = f'saved_runs/avg_effects/{timestamp}_run'
os.makedirs(f'{prefix}')
print(prefix)

model_types = ['gb', 'tree', 'log', 'rf', 'xgb']

params_dict = {
    'model_types': model_types,
    'schools': schools,
    'num_its': num_its
}

with open(f'{prefix}/params.pkl', 'wb') as f:
    pickle.dump(params_dict, f)


all_tasks = []
curr_seed = 1234

for m_index in range(len(model_types)):
    for num_schools_index in range(len(schools)):
        for i in range(num_its):
            all_tasks.append((m_index, num_schools_index, i, curr_seed))
            curr_seed += 1

results = Parallel(n_jobs = max(multiprocessing.cpu_count() - 1, 1))(
    delayed(enroll_and_calc_stats)(schools[num_schools_index], enroll_X, enroll_y, capacity, model_types[m_index], i, seed=seed, identifier = (m_index, num_schools_index, i)) 
    for m_index, num_schools_index, i, seed in tqdm(all_tasks, desc = 'Running enrollment simulations')
)

results_arr = np.zeros((len(model_types), len(schools), num_its, 4))

for identifier, stats in tqdm(results, desc = "Collecting Results"):
    model_type_index, school_num_index, iteration = identifier
    results_arr[model_type_index, school_num_index, iteration] = np.array(stats)

with open(f'{prefix}/stats.pkl', 'wb') as f:
    pickle.dump(results_arr, f)



saved_runs/avg_effects/20251217_190807_run


Running enrollment simulations:   0%|          | 0/40000 [00:00<?, ?it/s]

# Enrollment Simulation 2 of 3: Group-Aware Admissions Policies

This cell generates and saves simulation data tracking the difference in average student quality between enrolled classes in monoculture and polyculture. See *Enrollment Outcomes when Accepting Based on Groups* in Section 5.2.2 of the Arxiv manuscript for associated discussion. Results can be visualized in `simulation_plots.ipynb`.

In [16]:
schools = [1, 2, 4, 8]
capacity = 10000
num_its = 10
group_fraction = 0.25
groups = ['synth', 'sport']

os.makedirs('saved_runs/group_effects/synth_groups', exist_ok = True)
os.makedirs('saved_runs/group_effects/sport_groups', exist_ok = True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
synth_prefix = f'saved_runs/group_effects/synth_groups/{timestamp}_run'
sport_prefix = f'saved_runs/group_effects/sport_groups/{timestamp}_run'
os.makedirs(f'{synth_prefix}')
os.makedirs(f'{sport_prefix}')
print(synth_prefix)
print(sport_prefix)

model_types = ['gb', 'tree', 'log', 'rf', 'xgb']

params_dict = {
    'model_types': model_types,
    'schools': schools,
    'num_its': num_its, 
    'group_type': 'sport',
    'capacity': capacity
}

with open(f'{sport_prefix}/params.pkl', 'wb') as f:
    pickle.dump(params_dict, f)

params_dict['group_type'] = 'synth'
params_dict['group_fraction'] = group_fraction

with open(f'{synth_prefix}/params.pkl', 'wb') as f:
    pickle.dump(params_dict, f)



all_tasks = []
curr_seed = 1234 + 1000000

for group_index in range(2):
    for m_index in range(len(model_types)):
        for num_schools_index in range(len(schools)):
            for i in range(num_its):
                all_tasks.append((group_index, m_index, num_schools_index, i, curr_seed))
                curr_seed += 1

results = Parallel(n_jobs = max(multiprocessing.cpu_count() - 1, 1))(
    delayed(enroll_and_calc_stats)(schools[num_schools_index], enroll_X, enroll_y, capacity, model_types[m_index], i, groups=groups[group_index], group_fraction = group_fraction, seed=seed, identifier = (group_index, m_index, num_schools_index, i)) 
    for group_index, m_index, num_schools_index, i, seed in tqdm(all_tasks, desc = 'Running enrollment simulations')
)

synth_results_arr = np.zeros((len(model_types), len(schools), num_its, 4))
sport_results_arr = np.zeros((len(model_types), len(schools), num_its, 4))

for identifier, stats in tqdm(results, desc = "Collecting Results"):
    group_index, model_type_index, school_num_index, iteration = identifier
    if group_index == 0:
        synth_results_arr[model_type_index, school_num_index, iteration] = np.array(stats)
    else:
        assert(group_index == 1)
        sport_results_arr[model_type_index, school_num_index, iteration] = np.array(stats)


with open(f'{synth_prefix}/stats.pkl', 'wb') as f:
    pickle.dump(synth_results_arr, f)


with open(f'{sport_prefix}/stats.pkl', 'wb') as f:
    pickle.dump(sport_results_arr, f)



saved_runs/group_effects/synth_groups/20251217_195714_run
saved_runs/group_effects/sport_groups/20251217_195714_run


Running enrollment simulations:   0%|          | 0/400 [00:00<?, ?it/s]

# Enrollment Simulation 3 of 3: Impact of Student Preference Information on Student Enrollment

This cell generates and saves simulation data tracking the difference in average student quality between enrolled classes in monoculture and polyculture. See *“Winner’s Curse” Effects* in Section 5.2.2 of the Arxiv manuscript for associated discussion. Results can be visualized in `simulation_plots.ipynb`.

In [17]:
schools = [1, 2, 4, 8]
capacity = 10000
num_its = 2000

os.makedirs('saved_runs/legacy_effects', exist_ok = True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
prefix = f'saved_runs/legacy_effects/{timestamp}_run'
os.makedirs(f'{prefix}')
print(prefix)

model_types = ['gb', 'tree', 'log', 'rf', 'xgb']

params_dict = {
    'model_types': model_types,
    'schools': schools,
    'num_its': num_its
}

with open(f'{prefix}/params.pkl', 'wb') as f:
    pickle.dump(params_dict, f)


all_tasks = []
curr_seed = 1234 + 2000000

for m_index in range(len(model_types)):
    for num_schools_index in range(len(schools)):
        for i in range(num_its):
            all_tasks.append((m_index, num_schools_index, i, curr_seed))
            curr_seed += 1

results = Parallel(n_jobs = max(multiprocessing.cpu_count() - 1, 1))(
    delayed(enroll_and_calc_stats)(schools[num_schools_index], enroll_X, enroll_y, capacity, model_types[m_index], i, groups='legacy', seed=seed, identifier = (m_index, num_schools_index, i)) 
    for m_index, num_schools_index, i, seed in tqdm(all_tasks, desc = 'Running legacy enrollment simulations')
)

results_arr = np.zeros((len(model_types), len(schools), num_its, 4))

for identifier, stats in tqdm(results, desc = "Collecting Results"):
    model_type_index, school_num_index, iteration = identifier
    results_arr[model_type_index, school_num_index, iteration] = np.array(stats)

with open(f'{prefix}/stats.pkl', 'wb') as f:
    pickle.dump(results_arr, f)

saved_runs/legacy_effects/20251217_195746_run


Running legacy enrollment simulations:   0%|          | 0/40000 [00:00<?, ?it/s]